# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [92]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [93]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,17.23,36,55,2.68,US,1713914729
1,1,avarua,-21.2078,-159.7750,28.03,61,40,3.09,CK,1713914792
2,2,port-aux-francais,-49.3500,70.2167,4.33,69,45,16.28,TF,1713914793
3,3,la passe,45.5549,-0.8967,11.10,80,66,3.79,FR,1713914794
4,4,adamstown,-25.0660,-130.1015,23.00,85,100,8.87,PN,1713914794


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [94]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "OSM", 
    frame_width = 600,
    frame_height = 400, 
    alpha= 0.6,
    size = "Humidity",
    color = "City")

# Display the map
city_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [95]:
# Narrow down cities that fit criteria and drop any results with null values
fave_cities = city_data_df.loc[(city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 28) & (city_data_df["Humidity"] < 60)]

# Drop any rows with null values
fave_cities = fave_cities.dropna()

# Display sample data
fave_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,laguna,38.4210,-121.4238,21.75,56,75,4.12,US,1713914819
25,25,izki,22.9333,57.7667,25.34,55,16,1.70,OM,1713914824
29,29,barah,13.7000,30.3667,26.64,10,12,3.41,SD,1713914826
66,66,baabda,33.8339,35.5442,27.01,30,97,2.56,LB,1713914867
75,75,essaouira,31.5125,-9.7700,23.72,27,24,4.12,MA,1713914878


### Step 3: Create a new DataFrame called `hotel_df`.

In [97]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = fave_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
21,laguna,US,38.4210,-121.4238,56,
25,izki,OM,22.9333,57.7667,55,
29,barah,SD,13.7000,30.3667,10,
66,baabda,LB,33.8339,35.5442,30,
75,essaouira,MA,31.5125,-9.7700,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [98]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "format": "json",
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
laguna - nearest hotel: Holiday Inn Express & Suites
izki - nearest hotel: No hotel found
barah - nearest hotel: No hotel found
baabda - nearest hotel: granda house
essaouira - nearest hotel: Essaouira Wind Palace
illizi - nearest hotel: بوناقة
sterling - nearest hotel: Residence Inn by Marriott Dulles Airport at Dulles 28 Centre
kheralu - nearest hotel: No hotel found
elko - nearest hotel: No hotel found
geraldton - nearest hotel: Ocean Centre Hotel
turpan - nearest hotel: Oriental
karratha - nearest hotel: Karratha International Hotel
yangiyer - nearest hotel: No hotel found
biloela - nearest hotel: Settlers Hotel Motel
at taj - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
rajula - nearest hotel: Hotel Akshardham
portland - nearest hotel: the Hoxton
al artawiyah - nearest hotel: No hotel found
south hedland - nearest hotel: Port Haven
kutum - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
harran - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
21,laguna,US,38.4210,-121.4238,56,Holiday Inn Express & Suites
25,izki,OM,22.9333,57.7667,55,No hotel found
29,barah,SD,13.7000,30.3667,10,No hotel found
66,baabda,LB,33.8339,35.5442,30,granda house
75,essaouira,MA,31.5125,-9.7700,27,Essaouira Wind Palace
106,illizi,DZ,26.4833,8.4667,25,بوناقة
107,sterling,US,39.0062,-77.4286,31,Residence Inn by Marriott Dulles Airport at Du...
130,kheralu,IN,23.8833,72.6167,57,No hotel found
153,elko,US,41.0002,-115.5012,30,No hotel found
161,geraldton,AU,-28.7667,114.6000,37,Ocean Centre Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [99]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400, 
    hover_cols = "Hotel Name",
    alpha= 0.6,
    size = "Humidity",
    color = "City")

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)